In [19]:
import random
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [20]:
USER_AGENTS = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]

In [21]:
#设置北京各区域对应的网页url
region = {
    'dongcheng': 'dongcheng/',
    'xicheng': 'xicheng/',
    'chaoyang': 'chaoyang/',
    'haidian': 'haidian/',
    'fengtai': 'fengtai/',
    'shijingshan': 'shijingshan/',
    'tongzhou': 'tongzhou/',
    'changping': 'changping/',
    'daxing': 'daxing/',
    'yizhuangkaifaqu': 'yizhuangkaifaqu/',
    'shunyi': 'shunyi/',
    'mentougou': 'mentougou/',
    'fangshan': 'fangshan/',
    'pinggu': 'pinggu/',
    'huairou': 'huairou/',
    'miyun': 'miyun/',
    'yanqing': 'yanqing/',
}

In [22]:
#随机取user-agents
headers = {"User-Agent": random.choice(USER_AGENTS)}

In [23]:
def spider(regions):
    main_url = 'https://bj.fang.ke.com/loupan/'
    for key, values in region.items():
        if key == regions:
            all_list = []
            url = main_url + values
            response = requests.get(url, timeout=10, headers=headers)
            html = response.content
            soup = BeautifulSoup(html, "lxml")
            #通过得到的结果计算页数，每页10个，并进行四舍五入
            if soup.find('span', class_="value") == None: 
                break
            page = round(int(soup.find('span', class_="value").string) / 10)
            for i in range(1, page + 1):
                sleep(1.7)
                page_url = main_url + values + f'pg{i}'
                response = requests.get(page_url, timeout=10, headers=headers)
                html = response.content
                soup = BeautifulSoup(html, "lxml")
                #发现网页在到达50多页的时候不会出现数据，防报错
                try:
                    house_elements = soup.find_all('li', class_="resblock-list post_ulog_exposure_scroll has-results")
                    pass
                    for house_elem in house_elements:
                        #房价
                        price = house_elem.find('span', class_="number")
                        #提取是否存在支持vr看房的，支持为1，不支持为0
                        try:
                            desc = house_elem.find('li', class_="icon vr vr-animation-forever").text
                            if desc == "":
                                have_vr = 1
                        except Exception as e:
                            have_vr = 0
                        #总价的阈值
                        total = house_elem.find('div', class_="second")
                        #楼盘的名称
                        loupan = house_elem.find('a', class_='name')
                        # 清理数据，去除空白文本和无用的中文计量单位
                        try:
                            price = price.text.strip()
                        except Exception as e:
                            price = '0'
                        loupan = loupan.text.replace("\n", "")
                        try:
                            total = total.text.strip().replace(u'总价', '')
                            total = total.replace(u'/套起', '').replace('(万/套)', '')
                        except Exception as e:
                            total = '0'
                        #数据装入列表
                        data = loupan, price, total, have_vr
                        all_list.append(data)
                except:
                    break
            #通过pandas保存为csv文件
            df = pd.DataFrame(all_list)
            df.to_csv(f"{key}.csv", index=False,encoding="utf_8_sig")
            print(f"{key}.csv保存完毕")
        else:
            if regions == 'all':
                all_list = []
                url = main_url + values
                response = requests.get(url, timeout=10, headers=headers)
                html = response.content
                soup = BeautifulSoup(html, "lxml")
                if soup.find('span', class_="value") == None: 
                    break
                page = round(int(soup.find('span', class_="value").string) / 10)
                for i in range(1, page + 1):
                    sleep(1.7)
                    page_url = main_url + values + f'pg{i}'
                    response = requests.get(page_url, timeout=10, headers=headers)
                    html = response.content
                    soup = BeautifulSoup(html, "lxml")
                    try:
                        house_elements = soup.find_all('li',
                                                       class_="resblock-list post_ulog_exposure_scroll has-results")
                        pass
                        for house_elem in house_elements:
                            price = house_elem.find('span', class_="number")
                            try:
                                desc = house_elem.find('li', class_="icon vr vr-animation-forever").text
                                if desc == "":
                                    have_vr = 1
                            except Exception as e:
                                have_vr = 0
                            total = house_elem.find('div', class_="second")
                            loupan = house_elem.find('a', class_='name')
                            # 继续清理数据
                            try:
                                price = price.text.strip()
                            except Exception as e:
                                price = '0'
                            loupan = loupan.text.replace("\n", "")
                            try:
                                total = total.text.strip().replace(u'总价', '')
                                total = total.replace(u'/套起', '').replace('(万/套)', '')
                            except Exception as e:
                                total = '0'
                            data = loupan, price, total, have_vr
                            all_list.append(data)
                    except:
                        break
                df = pd.DataFrame(all_list)
                df.to_csv(f"{key}.csv", index=False, encoding="utf_8_sig")
                print(f"{key}.csv保存完毕")

In [24]:
spider('all')

dongcheng.csv保存完毕
xicheng.csv保存完毕
chaoyang.csv保存完毕
haidian.csv保存完毕
fengtai.csv保存完毕
shijingshan.csv保存完毕
tongzhou.csv保存完毕
changping.csv保存完毕
daxing.csv保存完毕
yizhuangkaifaqu.csv保存完毕
shunyi.csv保存完毕
mentougou.csv保存完毕
fangshan.csv保存完毕
pinggu.csv保存完毕
huairou.csv保存完毕
miyun.csv保存完毕
yanqing.csv保存完毕
